In [9]:
import re
from datetime import datetime, timedelta
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib
import os

# Carregar modelo de linguagem português
try:
    nlp = spacy.load("pt_core_news_sm")
except:
    os.system("python -m spacy download pt_core_news_sm")
    nlp = spacy.load("pt_core_news_sm")
# Controle para reprocessar modelo
REPROCESSAR_MODELO = False

# Treinar ou carregar modelo supervisionado
def treinar_modelo_supervisionado(caminho_csv='/content/mensagens_treinamento_csv.csv', salvar_em='modelo_logreg.pkl'):
    df = pd.read_csv(caminho_csv)
    df = df[df['mensagem'].notnull()]
    textos = df['mensagem'].astype(str).tolist()
    labels = df['valida'].astype(int).tolist()
    vetorizador = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), max_features=1000)

    X = vetorizador.fit_transform(textos)
    modelo = LogisticRegression()
    modelo.fit(X, labels)
    joblib.dump((vetorizador, modelo), salvar_em)
    print(f"Modelo salvo em: {salvar_em}")

# Ativar ou desativar o treinamento do modelo
if REPROCESSAR_MODELO:
    treinar_modelo_supervisionado()

# Tentar carregar modelo salvo
try:
    vectorizer, model = joblib.load('modelo_logreg.pkl')
except:
    print('⚠️ Modelo supervisionado não encontrado. Usando conjunto mínimo.')
    treinamento_textos = [
        "passando na feira", "chegou na praça", "tá na ponte", "15a na feira", "saiu da praça",
        "ele passa umas 12:00 na praça", "quando ele sai da feira?", "normalmente passa 13:10",
        "tô aqui esperando", "alguém viu o ônibus?"
    ]
    treinamento_labels = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000)
    X = vectorizer.fit_transform(treinamento_textos)
    model = LogisticRegression()
    model.fit(X, treinamento_labels)

# Função de predição com modelo treinado
def mensagem_valida_ml(mensagem):
    if pd.isnull(mensagem):
        return False
    X_test = vectorizer.transform([mensagem])
    return model.predict(X_test)[0] == 1

# Processar mensagens reais do WhatsApp
mensagens_extraidas = []
data_extraidas = []
hora_extraidas = []
telefone_extraidas = []
regex_mensagem = re.compile(r"(\d{2}/\d{2}/\d{4}) (\d{2}:\d{2}) - ([^:]+): (.+)")

with open("/content/conversa_unificada 14 -24 de maio.txt", 'r', encoding='utf-8') as f:
#with open("/content/Conversa_Unificada_Onde_esta_onibus_UFOB_Online.txt", 'r', encoding='utf-8') as f:

    for linha in f:
        match = regex_mensagem.match(linha.strip())
        if match:
            data, hora, telefone, mensagem = match.groups()
            mensagens_extraidas.append(mensagem.strip())
            data_extraidas.append(data)
            hora_extraidas.append(hora)
            telefone_extraidas.append(telefone)

# Criar DataFrame com metadados extraídos
df_processado = pd.DataFrame({
    "data": data_extraidas,
    "hora": hora_extraidas,
    "telefone": telefone_extraidas,
    "mensagem": mensagens_extraidas
})

df_processado = df_processado[df_processado['mensagem'].notnull()]
df_processado['é válida?'] = df_processado['mensagem'].apply(mensagem_valida_ml)
df_validas = df_processado[df_processado['é válida?'] == True].copy()

def extrair_ponto(mensagem):
    pontos = {
        'feira': ['feira', 'perto da feira'],
        'assaí': ['assai', 'açai', 'acai', 'assaí'],
        'praça das corujas': ['praça', 'praça das corujas'],
        'ufob': ['ufob'],
    }
    for ponto, padroes in pontos.items():
        for p in padroes:
            if p in mensagem.lower():
                return ponto
    return None

def definir_linha(ponto, mensagem):
    if ponto in ['assaí', 'feira']:
        return '15'
    if ponto in ['ufob', 'praça das corujas']:
        if '15a' in mensagem.lower():
            return '15A'
        return '15A' if 'a' in mensagem.lower() else '15'
    return '15'

# Horários de referência (exemplo reduzido)
horarios_linha_15 = {
    'vila nova': ['06:20', '06:35', '08:05', '09:45', '11:40', '13:10', '13:50', '15:20', '16:35', '18:00', '19:10', '21:30'],
    'pérola': ['06:24', '06:39', '08:09', '09:49', '11:44', '13:14', '13:54', '15:24', '16:39', '18:04', '19:14', '21:34'],
    'burguer king': ['06:26', '06:41', '08:11', '09:51', '11:46', '13:16', '13:56', '15:26', '16:41', '18:06', '19:16', '21:36'],
    'assaí': ['06:28', '06:43', '08:13', '09:53', '11:48', '13:18', '13:58', '15:28', '16:43', '18:08', '19:18', '21:38'],
    'vila shop': ['06:30', '06:45', '08:15', '09:55', '11:50', '13:20', '14:00', '15:30', '16:45', '18:10', '19:20', '21:40'],
    'feira': ['06:33', '06:48', '08:18', '09:58', '11:53', '13:23', '14:03', '15:33', '16:48', '18:13', '19:23', '21:43'],
    'praça das corujas': ['06:36', '06:51', '08:21', '10:01', '11:56', '13:26', '14:06', '15:36', '16:51', '18:16', '19:26', '21:46']
}

horarios_linha_15A = {
    'praça das corujas': ['06:20', '06:45', '07:00', '08:00', '09:40', '10:30', '12:00', '13:10', '13:50', '15:00', '16:10', '18:00', '19:30'],
    'correios': ['06:32', '06:57', '07:12', '08:12', '09:52', '10:42', '12:12', '13:22', '14:02', '15:12', '16:22', '18:12', '19:42'],
    'barreirinhas': ['06:36', '07:01', '07:16', '08:16', '09:56', '10:46', '12:16', '13:26', '14:06', '15:16', '16:26', '18:16', '19:46'],
    'ufob': ['06:45', '07:10', '07:25', '08:25', '10:05', '10:55', '12:25', '13:35', '14:15', '15:25', '16:35', '18:25', '19:55']
}

def encontrar_horario_e_atraso(data_str, hora_str, ponto, linha):
    if ponto is None or linha is None:
        return None, None
    horarios = horarios_linha_15.get(ponto) if linha == '15' else horarios_linha_15A.get(ponto)
    if not horarios:
        return None, None
    dt_msg = datetime.strptime(f"{data_str} {hora_str}", "%d/%m/%Y %H:%M")
    for h in reversed(horarios):
        hora_teorica = datetime.strptime(f"{data_str} {h}", "%d/%m/%Y %H:%M")
        if dt_msg >= hora_teorica:
            atraso = int((dt_msg - hora_teorica).total_seconds() / 60)
            return h, atraso
    return horarios[0], int((dt_msg - datetime.strptime(f"{data_str} {horarios[0]}", "%d/%m/%Y %H:%M")).total_seconds() / 60)

# Aplicar regras e construir tabela
resultados = []
for _, row in df_validas.iterrows():
    ponto = extrair_ponto(row['mensagem'])
    linha = definir_linha(ponto, row['mensagem'])
    horario_ideal, atraso = encontrar_horario_e_atraso(row['data'], row['hora'], ponto, linha)
    dt_full = datetime.strptime(f"{row['data']} {row['hora']}", "%d/%m/%Y %H:%M")
    resultados.append({
        'data': row['data'],
        'hora': row['hora'],
        'telefone': row['telefone'],
        'mensagem': row['mensagem'],
        'ponto': ponto,
        'linha': linha,
        'horario_ideal': horario_ideal,
        'atraso_minutos': atraso,
        'datetime': dt_full
    })

df_resultado = pd.DataFrame(resultados)

def gerar_resumo_atrasos(df):
    df = df.sort_values(by='datetime').reset_index(drop=True)
    nova_tabela = []
    ultimo_horario = {}
    for _, row in df.iterrows():
        chave = row['linha']
        horario_atual = row['datetime']
        if chave in ultimo_horario:
            tempo_diff = (horario_atual - ultimo_horario[chave]['datetime']).total_seconds() / 60
            if tempo_diff < 30:
                if row['atraso_minutos'] > ultimo_horario[chave]['atraso_minutos']:
                    ultimo_horario[chave] = row
                continue
            else:
                nova_tabela.append(ultimo_horario[chave])
        ultimo_horario[chave] = row
    nova_tabela.extend(ultimo_horario.values())
    return pd.DataFrame(nova_tabela)[['data', 'hora', 'linha', 'atraso_minutos']]

resumo_df = gerar_resumo_atrasos(df_resultado)

try:
    import ace_tools as tools
    tools.display_dataframe_to_user(name="Tabela Detalhada com Atrasos", dataframe=df_resultado)
    tools.display_dataframe_to_user(name="Resumo de Atrasos por Linha", dataframe=resumo_df)

except:
    #print(df_resultado)
    #print(resumo_df)

    df_resultado.to_csv('relatorio_atrasos_onibus_IA.csv', index=False)
    resumo_df.to_csv('resumo_maiores_atrasos_IA.csv', index=False)
    display(df_resultado)
    display(resumo_df)


,data,hora,telefone,mensagem,ponto,linha,horario_ideal,atraso_minutos,datetime
0,14/05/2025,10:22,+55 77 9800-7719,vai passar agora na praça,praça das corujas,15A,09:40,42.0,2025-05-14 10:22:00
1,14/05/2025,10:28,+55 77 9839-8067,"Passando na feira agora, tá como 15 A.",feira,15,09:58,30.0,2025-05-14 10:28:00
2,14/05/2025,12:10,+55 77 9173-8851,Chegou na feira ✨,feira,15,11:53,17.0,2025-05-14 12:10:00
3,14/05/2025,13:22,+55 77 9845-5233,chegou na praça agr,praça das corujas,15A,13:10,12.0,2025-05-14 13:22:00
4,14/05/2025,13:36,+55 77 9962-6321,Já chegou na Ufob,ufob,15A,13:35,1.0,2025-05-14 13:36:00
...,...,...,...,...,...,...,...,...,...
82,23/05/2025,11:55,+55 77 9964-4384,Na praça das corujas,praça das corujas,15A,10:30,85.0,2025-05-23 11:55:00
83,23/05/2025,16:32,+55 77 9952-2285,15A saindo da UFOB,ufob,15A,15:25,67.0,2025-05-23 16:32:00
84,23/05/2025,16:33,+55 88 9400-7611,15saindo da praça,praça das corujas,15A,16:10,23.0,2025-05-23 16:33:00
85,23/05/2025,18:32,+55 63 9272-6116,15A passando na feira,feira,15,18:13,19.0,2025-05-23 18:32:00


,data,hora,linha,atraso_minutos
0,03/04/2025,07:17,15A,17.0
1,04/04/2025,07:15,15,32.0
2,04/04/2025,07:36,15A,36.0
3,04/04/2025,08:40,15,22.0
4,04/04/2025,11:56,15A,86.0
...,...,...,...,...
82,23/05/2025,11:55,15A,85.0
81,23/05/2025,08:39,15,21.0
83,23/05/2025,16:32,15A,67.0
86,24/05/2025,07:10,15A,10.0


In [ ]:
from matplotlib import pyplot as plt
df_resultado['atraso_minutos'].plot(kind='line', figsize=(8, 4), title='atraso_minutos')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
df_resultado['atraso_minutos'].plot(kind='line', figsize=(8, 4), title='atraso_minutos')
plt.gca().spines[['top', 'right']].set_visible(False)